In [3]:
!sudo apt install tesseract-ocr tesseract-ocr-rus # Tesseract with lrussian support
!pip install pytesseract pillow opencv-python pdf2image ocrfixr fpdf transformers torch hashlib

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr-rus is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
ERROR: Ignored the following yanked versions: 20081119
ERROR: Could not find a version that satisfies the requirement hashlib (from versions: none)
ERROR: No matching distribution found for hashlib


In [2]:
from fpdf import FPDF
import cv2
import pytesseract
from pdf2image import convert_from_path
import numpy as np
from ocrfixr import spellcheck
import os
from transformers import MarianMTModel, MarianTokenizer
import hashlib
pytesseract.pytesseract.tesseract_cmd = r'/bin/tesseract'



def process_image(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
    processed_image = cv2.morphologyEx(gray_image, cv2.MORPH_OPEN, kernel)

    inverted_image = 255 - processed_image

    custom_config = r'--oem 3 --psm 6 -l rus'

    extracted_text = pytesseract.image_to_string(inverted_image, config=custom_config)

    return extracted_text

def correct_ocr_text(text):
    corrected_text = spellcheck(text).fix()
    return corrected_text

def full_process_and_clean(input_file):
    total_text = ""

    if input_file.lower().endswith('.pdf'):
        images = convert_from_path(input_file)
        for img in images:
            open_cv_image = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
            extracted_text = process_image(open_cv_image)
            corrected_text = correct_ocr_text(extracted_text)
            total_text += corrected_text + "\n"
    else:

        image = cv2.imread(input_file)
        extracted_text = process_image(image)
        corrected_text = correct_ocr_text(extracted_text)
        total_text += corrected_text

    return total_text

def save_to_pdf(text, outputdir="."):
    sha1_hash = lambda text: hashlib.sha1(text.encode()).hexdigest()
    output_filename = f"TEXT_{sha1_hash(text)}.pdf"
    output_path = os.path.join(outputdir, output_filename)
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, text)
    pdf.output(output_path, "F")

def translate(text: str) -> str:
    model_name = "Helsinki-NLP/opus-mt-ru-en"
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)

    translated = model.generate(**tokenizer(text, return_tensors="pt", padding=True))

    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)

    return translated_text

def main(input_file, outdir="."):
    return translate(full_process_and_clean(input_file))





/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:

if __name__ == "__main__":
  input_file = "russian text.pdf"
  OCRed_text = main(input_file)
  save_to_pdf(OCRed_text, outputdir=".")
  print(OCRed_text)